In [7]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from ipyparallel import Client
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.interfaces.fsl import ExtractROI

%matplotlib inline

In [9]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
acq_params = home_dir + 'scripts/acqparams.txt'
os.chdir(home_dir)

#get subject list
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))

In [32]:
##merge the calibration scans
def merge_scans(sub):
    home_dir = '/home/iballard/fd/'
    cal_dir = home_dir + 'data/' + sub + '/cal/'
    out_dir = cal_dir + 'lyman/'
                              
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    for i in range(1,4+1):

        out_f = out_dir + '/merged_' + str(i)
                             
        cal1 = cal_dir + 'cal1_' + str(i) + '.nii.gz'
        cal2 = cal_dir + 'cal2_' + str(i) + '.nii.gz'
        
        out_cal1 = cal_dir + 'cal1_' + str(i) + '_slice.nii.gz'
        out_cal2 = cal_dir + 'cal2_' + str(i) + '_slice.nii.gz'

        cmd = ['fslroi',cal1,out_cal1,'1','1']
        os.system(' '.join(cmd))

        cmd = ['fslroi',cal2,out_cal2,'1','1']
        os.system(' '.join(cmd))      
        
        cmd = ['fslmerge','-t',out_f,out_cal2,out_cal1]
        os.system(' '.join(cmd))
    return out_dir

In [ ]:
#run analysis
pool = multiprocessing.Pool(processes = 15)
pool.map(merge_scans,sub_list)
pool.terminate()
pool.join()

In [33]:
#assign to correct run and motion correct (note that this wasn't necessary, it turns out)
def create_fm(scan_tuple):
    sub, exp, run = scan_tuple

    home_dir = '/home/iballard/fd/'
    out_dir = home_dir + 'data/' + sub + '/cal/lyman/'
    data_f = home_dir + 'data/' + sub + '/func/' + exp + '/run_' + str(run) + '.nii.gz'

    if os.path.exists(data_f):
        if exp == 'loc': #localizers were run last
            cal_scan = 4
        elif run == 1:
            cal_scan = 1
        elif run == 2:
            cal_scan = 2
        elif run == 3:
            cal_scan = 3

        #for this subject, run2 cal scans are junk
        if sub == 'fd_101' and run == 2 :
            cal_scan = 1

        #one subject has extra cal scan for 2nd localizer run on a different day
        if sub == 'fd_115' and exp == 'loc' and run == 2:
            cal_scan = 5

        if sub == 'fd_127' and exp == 'sim' and run == 2:
            cal_scan = 2 #one is messed up

        #get correct field inputs
        cal_scan = out_dir + '/merged_' + str(cal_scan)
        out_cal = out_dir + '/fm_' + exp + '_' + str(run) + '_mc'
        
        if not os.path.exists(out_cal):
            #run motion correction
            out_f = out_dir + '/' + exp + str(run) + '_slice.nii.gz'
            idx = (nib.load(data_f).shape[-1] - 8)//2 #middle volume
            cmd = ['fslroi',data_f,out_f,str(idx),'1']
            os.system(' '.join(cmd))
            cmd = ['mcflirt','-in',cal_scan,'-reffile',out_f,'-out',out_cal]
            os.system(' '.join(cmd))    
            print ' '.join(cmd)

    return out_dir

In [16]:
in_tuples = []
for sub in #sub_list:
    for exp in ['loc']:
        for run in range(1,3):
            in_tuples.append((sub,exp,run))

In [16]:
dview.block = True
# dview.push(dict(home_dir=home_dir))
dview.execute("import numpy as np")
dview.execute("import nibabel as nib")
with dview.sync_imports():
    import numpy
    import os
    from os.path import abspath
    import nibabel as nib
out1 = dview.map_sync(create_fm,in_tuples)

importing numpy on engine(s)
importing os on engine(s)
importing abspath from os.path on engine(s)
importing nibabel on engine(s)


In [23]:
#run analysis
pool = multiprocessing.Pool(processes = 15)
pool.map(create_fm,in_tuples)
pool.terminate()
pool.join()

In [34]:
#just take 2 slices from fieldmap 
for scan_tuple in in_tuples:
    sub, exp, run = scan_tuple

    out_dir = os.path.abspath('data/' + sub + '/cal/lyman/')
    out_cal = out_dir + '/fm_' + exp + '_' + str(run) + '_mc.nii.gz'

    if os.path.exists(out_cal):

        tmp1 =  out_dir + '/' + sub + exp + str(run) + '_tmp1'
        tmp2 =  out_dir + '/' + sub + exp + str(run) + '_tmp2'
        tmp3 =  out_dir + '/fm_' + exp + '_' + str(run) + '_mc_pe0_pe1.nii.gz'
        print tmp3
        cmd = ['fslroi',out_cal,tmp1,'0','1']
        os.system(' '.join(cmd))
        
        cmd = ['fslroi',out_cal,tmp2,'1','1']
        os.system(' '.join(cmd))

        cmd = ['fslmerge','-t',tmp3,tmp2,tmp1]
        os.system(' '.join(cmd))
        
        os.remove(tmp1 + '.nii.gz')
        os.remove(tmp2 + '.nii.gz')


/data/home/iballard/fd/data/fd_115/cal/lyman/fm_loc_2_mc_pe0_pe1.nii.gz
